In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline, AutoConfig
import shap

In [2]:
def load_pipeline(INPUT_PATH):
    model = AutoModelForSequenceClassification.from_pretrained(INPUT_PATH)
    tokenizer = AutoTokenizer.from_pretrained(INPUT_PATH)
    config = AutoConfig.from_pretrained(INPUT_PATH)
    p = pipeline('text-classification', model=model, tokenizer=tokenizer, config=config, return_all_scores=True)
    return p

In [3]:
INPUT_PATH = '../../models/title'

pipeline_title = load_pipeline(INPUT_PATH)

In [4]:
INPUT_PATH = '../../models/title_summary'

pipeline_title_summary = load_pipeline(INPUT_PATH)

In [34]:
def try_input(pipeline, INPUT_TEXT, out=False):
    print(pipeline([INPUT_TEXT]))
    explainer = shap.Explainer(pipeline)
    shap_values = explainer([INPUT_TEXT])
    shap_values.output_names = ['CONTROVERSY', 'NO_CONTROVERSY']
    shap.plots.text(shap_values[0, :, 'CONTROVERSY']) # ES CONTROVERTIDO? 
    html_out = shap.plots.text(shap_values[0, :, 'CONTROVERSY'], display = False) # ES CONTROVERTIDO? 
    if out:
        file = open(INPUT_TEXT[:10].replace(' ', '_').lower()+".html","w")
        file.write(html_out)
        file.close()
    

In [35]:
INPUT_TEXT = "Guerra de Ucrania: Basta ya de mentiras, medias verdades y desinformación" # -- for adding the summary

html_out = try_input(pipeline_title, INPUT_TEXT, out=True)

[[{'label': 'LABEL_0', 'score': 0.31266161799430847}, {'label': 'LABEL_1', 'score': 0.6873384118080139}]]


  0%|          | 0/240 [00:00<?, ?it/s]

Partition explainer: 2it [00:17, 17.99s/it]                                                                                                                                                              


In [7]:
INPUT_TEXT = "Guerra de Ucrania: Basta ya de mentiras, medias verdades y desinformación -- El engaño de hoy será el lamento de mañana. Por ello, no solo hay que describir lo que está sucediendo desde múltiples puntos de vista, sino que también es necesario intentar alcanzar las razones por las que acontece. No es sencillo y, seguramente, no lo consiga debido a las múltiples limitaciones que poseo, pero es lo que he intentado hacer. Este es el resultado en quince puntos. No es una lectura cómoda ni corta, pero creo que necesaria. " # -- for adding the summary

try_input(pipeline_title_summary, INPUT_TEXT)

[[{'label': 'LABEL_0', 'score': 0.7468140721321106}, {'label': 'LABEL_1', 'score': 0.25318586826324463}]]


  0%|          | 0/248 [00:00<?, ?it/s]

Partition explainer: 2it [00:54, 54.03s/it]                                                                                                                                                              


In [8]:
# Features pushing the prediction higher are shown in red, those pushing the prediction lower are in blue.

# LABEL_0--> CONTROVERSY, LABEL_1 --> NO_CONTROVERSY